# Importation des Biblios
il faut installer tensorflow==1.14, et contractions, il faut enlever les commentaires de pip dans la cellule d 

In [0]:
#pip install contractions
#pip install tensorflow==1.14
import re
import pandas as pd
import string
import numpy as np
import contractions 
import matplotlib.pyplot as plt


import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem.porter import PorterStemmer
nltk.download('all')

stop_words = stopwords.words('english')
porter = PorterStemmer()

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import metrics
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.python.keras.layers import Dropout,Dense
from tensorflow.python.keras import Sequential
from sklearn.metrics import confusion_matrix
#from keras.models import Sequential
#from keras.layers import Dense,  Activation
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import numpy 
from sklearn import metrics
from scipy.stats import zscore
from keras.models import Sequential
from keras.layers import Dense,  Activation

#from tensorflow.python.keras.layers import Dense, Activation
#from tensorflow.python.keras import Sequential

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_

# Des fonctions Utils

In [0]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import ToktokTokenizer
from collections import defaultdict
token=ToktokTokenizer()
lemma=WordNetLemmatizer()

def remove_copyright(text):
  start0 = text.find("Copyright")
  start1 = text.find("©")
  if start0!=-1 & start1!=-1:
    start = min(start0, start1)
  elif start0!=-1 & start1==-1:
    start = start0
  elif start0==-1 & start1!=-1:
    start = start1
  else:
    start = -1
  if start!=-1: 
    text = text[0:start]
    
  return text

def remove_trialregistration(text):
  # The format for the ClinicalTrials.gov registry number is “NCT” followed by an 8-digit number
  text = re.sub('[N][C][T][[\d]{8}', '', text)
  text = re.sub('ClinicalTrials.gov', '', text)
  text = re.sub('Trial registration', '', text)
  text = re.sub('Trial register', '', text)
  text = re.sub('Identifier', '', text)
  
  return text

def remove_URL(text):
  url = re.compile(r'https?://\S+|www\.\S+')
  return url.sub(r'',str(text))
  

def classic_processing(text):
  text = remove_URL(text)
  text = remove_copyright(text)
  text = remove_trialregistration(text)
  text = text.translate(str.maketrans('', '', '/"()%-$@{}>=<[].;,:!?+&/#°®0123456789')).lower()
  text = text.strip() # Remove whitespaces
  text = text.split() # split the text to words
  text =  " ".join( text ) #Activer ce commentaire si vous voulez appliquer une contractions 
  return text

#************************************************* fonctions pour les types de tests *******************************

def replace_contractions(text):
  clean = classic_processing(text)
  return contractions.fix(clean)

#remove stop_words thanks to nltk, the Data must be in LowerCase + contractions.
stops = set(stopwords.words('english'))  
mylist = pd.read_csv("FrequentWords_v160320.csv")
stops2 = mylist['x'].tolist()

def remove_stop_words(text):
  words=token.tokenize(text)
  meaningful_words = [w for w in words if ((w.isalpha()==1) & (w not in stops) & (w not in stops2))]
  text_final =  " ".join(map(str, meaningful_words))
  return text_final

def lemitizeWords(text):
  words=token.tokenize(text)
  listLemma=[]
  for w in words:
    x=lemma.lemmatize(w, pos="v")
    listLemma.append(x)
  return ' '.join(map(str, listLemma))



#La stemmatisation = verbe infinitif
#les données doivent être tokenizées
def stemmatisation(text):
  words=token.tokenize(text)
  stemmed = [porter.stem(w) for w in words]
  return ' '.join(map(str,stemmed))

#************************************************* fonctions pour les types de classifieurs *******************************

def run_RNN(X):
  seed=7
  kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
  cvscores = []
  fold = 0
  for train, test in kfold.split(X):
    fold+=1
    
    x_train = X[train]
    y_train = y[train]
    x_test = X[test]
    y_test = y[test]

    model = Sequential()
    model.add(Dense(4, input_dim= X.shape[1], activation='relu')) 
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
  
    model.fit(x_train,y_train, epochs=10, batch_size=32, verbose= True)
  
    scores = model.evaluate(x_test,y_test, verbose=0)
    #print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
  
  return cvscores
    
 
def run_SVM (X):
  seed=7
  k_fold = KFold(n_splits=10, shuffle=True, random_state=seed)
  clf = SVC(C=1, cache_size=200, class_weight=None, coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear', max_iter=-1, probability=False, random_state=None, shrinking=True,tol=0.001, verbose=False) 
  scoring = 'accuracy'
  score = cross_val_score(clf, X, y, cv=k_fold, scoring=scoring)
  return score.mean()



# importer les données

In [0]:
#titre_brut
Data_pos = pd.read_csv('yes_not_clean.csv')
Data_neg = pd.read_csv('no_not_clean.csv',error_bad_lines=False , engine='python')
Neg = Data_neg[0:512] # extraire 512 observations non-INm pour avoir 1 équilibre 

Pos_titre = Data_pos[['Title','response']]
Neg_titre = Neg[['Title','response']]
Neg_titre['response'].replace({2: 0}, inplace=True) # remplacer la valeur dcu champ respone =2 par 0

df_yes_titre =Pos_titre
df_no_titre = Neg_titre

df_titre_yes = df_yes_titre.loc[:,['Title', 'response']]
df_titre_no = df_no_titre.loc[:,['Title', 'response']]
# récupération des X_yes et X_no , y_yes, y_no
array = df_titre_yes.values
X_yes = array[:,0] 
y_yes = array[:,1]
lengthX_yes=len(y_yes)
array = df_titre_no.values
X_no=array[:,0]
y_no = array[:,1]


X_no_short=X_no[0:lengthX_yes]
y_no_short=y_no[0:lengthX_yes]

# concatenation dans X des X_yes et X_no
X6=np.concatenate((X_yes, X_no_short))
y6=np.concatenate((y_yes, y_no_short))


#transformation des y en entier pour la classification
y=y6.astype('int')

#transformation des X en list de string
X6=np.array(X6).tolist()




/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


#Titre brut + traitement classique

In [0]:
list_classic_process = []
for text in X6:
  list_classic_process.append(classic_processing(text))
"""
vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(list_classic_process)
list_classic_process = vectors.toarray()
print ("RNN : titre_brut_trait_classic ",run_RNN(list_classic_process))
print ("SVM : titre_brut_trait_classic ",run_SVM(list_classic_process))
"""

'\nvectorizer = vectorizer = TfidfVectorizer()\nvectors = vectorizer.fit_transform(list_classic_process)\nlist_classic_process = vectors.toarray()\nprint ("RNN : titre_brut_trait_classic ",run_RNN(list_classic_process))\nprint ("SVM : titre_brut_trait_classic ",run_SVM(list_classic_process))\n'

# Titre Brut

In [0]:
vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X6)
X6 = vectors.toarray()
print ("RNN : titre_brut ",run_RNN(X6))
print ("SVM : titre_brut ",run_SVM(X6))

Epoch 1/10
919/919 [==============================] - 4s 4ms/step - loss: 0.6893 - acc: 0.6409
Epoch 2/10
919/919 [==============================] - 0s 81us/step - loss: 0.6718 - acc: 0.8803
Epoch 3/10
919/919 [==============================] - 0s 82us/step - loss: 0.6478 - acc: 0.9347
Epoch 4/10
919/919 [==============================] - 0s 84us/step - loss: 0.6200 - acc: 0.9565
Epoch 5/10
919/919 [==============================] - 0s 84us/step - loss: 0.5892 - acc: 0.9565
Epoch 6/10
919/919 [==============================] - 0s 89us/step - loss: 0.5562 - acc: 0.9608
Epoch 7/10
919/919 [==============================] - 0s 85us/step - loss: 0.5218 - acc: 0.9630
Epoch 8/10
919/919 [==============================] - 0s 88us/step - loss: 0.4869 - acc: 0.9652
Epoch 9/10
919/919 [==============================] - 0s 87us/step - loss: 0.4523 - acc: 0.9684
Epoch 10/10
919/919 [==============================] - 0s 87us/step - loss: 0.4188 - acc: 0.9674
Epoch 1/10
919/919 [====================

In [0]:
list_classic_process = []
for text in X6:
  list_classic_process.append(classic_processing(text))

vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X6)
X6 = vectors.toarray()
print ("RNN : titre_brut ",run_RNN(X6))
print ("SVM : titre_brut ",run_SVM(X6))

# Titre brut + traitement classic + replace contractions

In [0]:
titre_traite_replace_contractions = []
X26 = list_classic_process

for text in X26:
  titre_traite_replace_contractions.append(replace_contractions(text))

X27 = titre_traite_lemmatisation
vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X27)
X27 = vectors.toarray()
print ("RNN : titre_traite_replace_contractions ",run_RNN(X27))
print ("SVM : titre_traite_replace_contractions ",run_SVM(X27))

Epoch 1/10
919/919 [==============================] - 3s 4ms/step - loss: 0.6919 - acc: 0.5593
Epoch 2/10
919/919 [==============================] - 0s 82us/step - loss: 0.6828 - acc: 0.8052
Epoch 3/10
919/919 [==============================] - 0s 81us/step - loss: 0.6669 - acc: 0.8683
Epoch 4/10
919/919 [==============================] - 0s 79us/step - loss: 0.6465 - acc: 0.9097
Epoch 5/10
919/919 [==============================] - 0s 81us/step - loss: 0.6222 - acc: 0.9195
Epoch 6/10
919/919 [==============================] - 0s 90us/step - loss: 0.5947 - acc: 0.9282
Epoch 7/10
919/919 [==============================] - 0s 86us/step - loss: 0.5647 - acc: 0.9304
Epoch 8/10
919/919 [==============================] - 0s 85us/step - loss: 0.5329 - acc: 0.9347
Epoch 9/10
919/919 [==============================] - 0s 85us/step - loss: 0.5004 - acc: 0.9434
Epoch 10/10
919/919 [==============================] - 0s 89us/step - loss: 0.4684 - acc: 0.9445
Epoch 1/10
919/919 [====================

# Titre Brut + traitement classic + lemmatisation

In [0]:
titre_traite_lemmatisation = []
X24 = list_classic_process

for text in X24:
  titre_traite_lemmatisation.append(lemitizeWords(text))

X25 = titre_traite_lemmatisation
vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X25)
X25 = vectors.toarray()
print ("RNN : titre_traite_lemmatisation ",run_RNN(X25))
print ("SVM : titre_traite_lemmatisation ",run_SVM(X25))

Epoch 1/10
919/919 [==============================] - 3s 3ms/step - loss: 0.6894 - acc: 0.6474
Epoch 2/10
919/919 [==============================] - 0s 78us/step - loss: 0.6723 - acc: 0.8705
Epoch 3/10
919/919 [==============================] - 0s 80us/step - loss: 0.6474 - acc: 0.9314
Epoch 4/10
919/919 [==============================] - 0s 81us/step - loss: 0.6189 - acc: 0.9521
Epoch 5/10
919/919 [==============================] - 0s 80us/step - loss: 0.5884 - acc: 0.9608
Epoch 6/10
919/919 [==============================] - 0s 78us/step - loss: 0.5563 - acc: 0.9663
Epoch 7/10
919/919 [==============================] - 0s 80us/step - loss: 0.5232 - acc: 0.9663
Epoch 8/10
919/919 [==============================] - 0s 78us/step - loss: 0.4900 - acc: 0.9706
Epoch 9/10
919/919 [==============================] - 0s 84us/step - loss: 0.4572 - acc: 0.9695
Epoch 10/10
919/919 [==============================] - 0s 82us/step - loss: 0.4256 - acc: 0.9750
Epoch 1/10
919/919 [====================

# Titre Brut + traitement classic + remove stop words

In [0]:
titre_traite_remove_stop_words = []
X22 = list_classic_process

for text in X22:
  titre_traite_remove_stop_words.append(remove_stop_words(text))

X23 = titre_traite_remove_stop_words
vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X23)
X23 = vectors.toarray()
print ("RNN : titre_traite_remove_stop_words ",run_RNN(X23))
print ("SVM : titre_traite_remove_stop_words ",run_SVM(X23))

Epoch 1/10
919/919 [==============================] - 3s 3ms/step - loss: 0.6921 - acc: 0.5071
Epoch 2/10
919/919 [==============================] - 0s 78us/step - loss: 0.6847 - acc: 0.6420
Epoch 3/10
919/919 [==============================] - 0s 74us/step - loss: 0.6720 - acc: 0.7051
Epoch 4/10
919/919 [==============================] - 0s 77us/step - loss: 0.6535 - acc: 0.7835
Epoch 5/10
919/919 [==============================] - 0s 79us/step - loss: 0.6315 - acc: 0.8662
Epoch 6/10
919/919 [==============================] - 0s 85us/step - loss: 0.6068 - acc: 0.9032
Epoch 7/10
919/919 [==============================] - 0s 77us/step - loss: 0.5795 - acc: 0.9304
Epoch 8/10
919/919 [==============================] - 0s 76us/step - loss: 0.5508 - acc: 0.9532
Epoch 9/10
919/919 [==============================] - 0s 79us/step - loss: 0.5212 - acc: 0.9630
Epoch 10/10
919/919 [==============================] - 0s 77us/step - loss: 0.4911 - acc: 0.9684
Epoch 1/10
919/919 [====================

# Titre Brut + traitement classic +verbe infinitif

In [0]:
titre_traite_verb_infintif = []
X20 = list_classic_process

for text in X20:
  titre_traite_verb_infintif.append(stemmatisation(text))

X21 = titre_traite_verb_infintif
vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X20)
X21 = vectors.toarray()
print ("RNN : titre_traite_verb_infintif ",run_RNN(X21))
print ("SVM : titre_traite_verb_infintif ",run_SVM(X21))

Epoch 1/10
919/919 [==============================] - 2s 2ms/step - loss: 0.6893 - acc: 0.6083
Epoch 2/10
919/919 [==============================] - 0s 73us/step - loss: 0.6689 - acc: 0.8738
Epoch 3/10
919/919 [==============================] - 0s 75us/step - loss: 0.6379 - acc: 0.9336
Epoch 4/10
919/919 [==============================] - 0s 72us/step - loss: 0.6037 - acc: 0.9489
Epoch 5/10
919/919 [==============================] - 0s 78us/step - loss: 0.5693 - acc: 0.9510
Epoch 6/10
919/919 [==============================] - 0s 76us/step - loss: 0.5347 - acc: 0.9499
Epoch 7/10
919/919 [==============================] - 0s 76us/step - loss: 0.5000 - acc: 0.9587
Epoch 8/10
919/919 [==============================] - 0s 79us/step - loss: 0.4666 - acc: 0.9674
Epoch 9/10
919/919 [==============================] - 0s 77us/step - loss: 0.4340 - acc: 0.9728
Epoch 10/10
919/919 [==============================] - 0s 73us/step - loss: 0.4029 - acc: 0.9750
Epoch 1/10
919/919 [====================

# Titre Brut + replace contractions

In [0]:
#titre Brut replace contractions
titre_brut_replace_contractions = []

for text in X6:
  titre_brut_replace_contractions.append(replace_contractions(text))

X15 = titre_brut_replace_contractions

vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X15)
X15 = vectors.toarray()
print ("RNN : titre_brut_replace_contractions ",run_RNN(X15))
print ("SVM : titre_brut_replace_contractions ",run_SVM(X15))


Epoch 1/10
919/919 [==============================] - 1s 2ms/step - loss: 0.6914 - acc: 0.5941
Epoch 2/10
919/919 [==============================] - 0s 71us/step - loss: 0.6821 - acc: 0.7486
Epoch 3/10
919/919 [==============================] - 0s 67us/step - loss: 0.6668 - acc: 0.8346
Epoch 4/10
919/919 [==============================] - 0s 65us/step - loss: 0.6473 - acc: 0.8694
Epoch 5/10
919/919 [==============================] - 0s 77us/step - loss: 0.6239 - acc: 0.8934
Epoch 6/10
919/919 [==============================] - 0s 80us/step - loss: 0.5974 - acc: 0.9042
Epoch 7/10
919/919 [==============================] - 0s 73us/step - loss: 0.5686 - acc: 0.9140
Epoch 8/10
919/919 [==============================] - 0s 71us/step - loss: 0.5375 - acc: 0.9260
Epoch 9/10
919/919 [==============================] - 0s 72us/step - loss: 0.5059 - acc: 0.9369
Epoch 10/10
919/919 [==============================] - 0s 75us/step - loss: 0.4743 - acc: 0.9499
Epoch 1/10
919/919 [====================

# Titre brut + lemmatisation

In [0]:
# texte_brut_lemmatisation
titre_brut_lemmatisation = []

for text in X6:
  titre_brut_lemmatisation.append(lemitizeWords(text))

X88 = titre_brut_lemmatisation
vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X88)
X88 = vectors.toarray()
print ("RNN : titre_brut_lemmatisation ",run_RNN(X88))
print ("SVM : titre_brut_lemmatisation ",run_SVM(X88))

Epoch 1/10
919/919 [==============================] - 1s 1ms/step - loss: 0.6916 - acc: 0.5669
Epoch 2/10
919/919 [==============================] - 0s 67us/step - loss: 0.6786 - acc: 0.6507
Epoch 3/10
919/919 [==============================] - 0s 67us/step - loss: 0.6568 - acc: 0.7280
Epoch 4/10
919/919 [==============================] - 0s 77us/step - loss: 0.6307 - acc: 0.8107
Epoch 5/10
919/919 [==============================] - 0s 80us/step - loss: 0.6021 - acc: 0.8683
Epoch 6/10
919/919 [==============================] - 0s 79us/step - loss: 0.5714 - acc: 0.8977
Epoch 7/10
919/919 [==============================] - 0s 68us/step - loss: 0.5398 - acc: 0.9173
Epoch 8/10
919/919 [==============================] - 0s 71us/step - loss: 0.5074 - acc: 0.9304
Epoch 9/10
919/919 [==============================] - 0s 69us/step - loss: 0.4751 - acc: 0.9391
Epoch 10/10
919/919 [==============================] - 0s 72us/step - loss: 0.4434 - acc: 0.9467
Epoch 1/10
919/919 [====================

# titre brut + Remove stop words

In [0]:

#titre_brut_remove_stop_words 

titre_brut_remove_stop_words = []

for text in X6:
  titre_brut_remove_stop_words.append(remove_stop_words(text))

X2 = titre_brut_remove_stop_words

vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X2)
X2 = vectors.toarray()
print ("RNN : titre_brut_remove_stop_words ",run_RNN(X2))
print ("SVM : titre_brut_remove_stop_words ",run_SVM(X2))


Epoch 1/10
919/919 [==============================] - 1s 612us/step - loss: 0.6914 - acc: 0.5713
Epoch 2/10
919/919 [==============================] - 0s 57us/step - loss: 0.6819 - acc: 0.5930
Epoch 3/10
919/919 [==============================] - 0s 57us/step - loss: 0.6664 - acc: 0.5332
Epoch 4/10
919/919 [==============================] - 0s 72us/step - loss: 0.6470 - acc: 0.6322
Epoch 5/10
919/919 [==============================] - 0s 57us/step - loss: 0.6257 - acc: 0.6877
Epoch 6/10
919/919 [==============================] - 0s 64us/step - loss: 0.6029 - acc: 0.7497
Epoch 7/10
919/919 [==============================] - 0s 58us/step - loss: 0.5801 - acc: 0.7704
Epoch 8/10
919/919 [==============================] - 0s 59us/step - loss: 0.5562 - acc: 0.8477
Epoch 9/10
919/919 [==============================] - 0s 58us/step - loss: 0.5331 - acc: 0.8607
Epoch 10/10
919/919 [==============================] - 0s 58us/step - loss: 0.5102 - acc: 0.8857
Epoch 1/10
919/919 [==================

# Titre Brut + verb infinitif

In [0]:

#titre_brut_verbe_infinitif 

titre_brut_verb_infintif = []

for text in X6:
  titre_brut_verb_infintif.append(stemmatisation(text))

X1 = titre_brut_verb_infintif
vectorizer = vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X1)
X1 = vectors.toarray()
print ("RNN : titre_brut_verb_infintif ",run_RNN(X1))
print ("SVM : titre_brut_verb_infintif ",run_SVM(X1))







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
919/919 [==============================] - 0s 283us/step - loss: 0.6895 - acc: 0.6355
Epoch 2/10
919/919 [==============================] - 0s 56us/step - loss: 0.6744 - acc: 0.6855
Epoch 3/10
919/919 [==============================] - 0s 58us/step - loss: 0.6525 - acc: 0.7617
Epoch 4/10
919/919 [==============================] - 0s 56us/step - loss: 0.6273 - acc: 0.8509
Epoch 5/10
919/919 [==============================] - 0s 60us/step - loss: 0.5997 - acc: 0.8836
Epoch 6/10
919/919 [==============================] - 0s 57us/step - loss: 0.5701 - acc: 0.9260
Epoch 7/10
919/919 [==============================] - 0s 67us/step - loss: 0.5394 - acc: 0.9369
Epoch 8/10
919/919 [==============================] - 0s 71us/step - loss: 0.5081 - acc: 0.9499
Epoch 9/10
919/919 [==============================] - 0s 58us/step - loss: 0.4764 - acc: 0.9619
Epoch 10/10
919/919 [==============